In [ ]:
from pybaseball import batting_stats, playerid_reverse_lookup
import pandas as pd

In [31]:
def get_hitters_from_year(year, qual):
    hitter_data = batting_stats(year, qual=qual)

    hitter_ids = hitter_data["IDfg"].tolist()

    lookup_table = playerid_reverse_lookup(hitter_ids, key_type="fangraphs")

    missing_ids = [id for id in hitter_ids if id not in lookup_table["key_fangraphs"].values]

    trimmed_hitter_data = hitter_data[~hitter_data["IDfg"].isin(missing_ids)]

    # missing_players = hitter_data[hitter_data["IDfg"].isin(missing_ids)]

    # for _, row in missing_players.iterrows():
    #     print(f"Player {row['Name']} (ID: {row['IDfg']}) is missing")
    
    return lookup_table, trimmed_hitter_data

In [32]:
lookup_table, trimmed_hitter_data = get_hitters_from_year(2024, 100)

In [33]:
lookup_table.head()

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,edman,tommy,669242,edmat001,edmanto01,19470,2019.0,2024.0
1,schneider,davis,676914,schnd001,schneda03,23565,2023.0,2024.0
2,lopez,nicky,670032,lopen001,lopezni01,19339,2019.0,2024.0
3,lowe,brandon,664040,loweb001,lowebr01,18882,2018.0,2024.0
4,goodman,hunter,696100,goodh001,goodmhu01,29715,2023.0,2024.0


In [34]:
trimmed_hitter_data.head()

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
0,15640,2024,Aaron Judge,NYY,32,158,559,704,180,85,...,117.5,238,0.609,391,0.146,0.267,0.310,0.723,0.479,11.5
5,25764,2024,Bobby Witt Jr.,KCR,24,161,636,709,211,123,...,116.9,259,0.481,538,0.138,0.236,0.315,0.577,0.407,10.4
1,19755,2024,Shohei Ohtani,LAD,29,159,636,731,197,98,...,119.2,287,0.595,482,0.144,0.269,0.314,0.660,0.442,9.2
2,20123,2024,Juan Soto,NYY,25,157,576,713,166,90,...,115.7,262,0.568,461,0.189,0.263,0.316,0.646,0.462,7.9
16,26289,2024,Gunnar Henderson,BAL,23,159,630,719,177,102,...,113.1,256,0.539,475,0.167,0.265,0.283,0.492,0.374,8.2


In [ ]:
pitch_type_stats = ['IDfg', 'wFB', 'wSL', 'wCT', 'wCB', 'wCH', 'wSF', 'wKN']
hit_stats= ['IDfg', 'AVG+', 'OBP+', 'SLG+', 'BABIP+', 'BB%+', 'HardHit%']

,IDfg,Name
0,15640,Aaron Judge
6,25764,Bobby Witt Jr.
2,19755,Shohei Ohtani
3,20123,Juan Soto
19,26289,Gunnar Henderson
